### 1.1 二叉搜索树模板

#### 一. 二叉搜索树的结构
1. 二叉搜索树处处满足顺序性 :   
  任一节点$r$的左(右)子树中, 所有节点(若存在)均不大于(不小于)$r$ 
2. 若按照"左子树<父节点<右子树"的顺序组织节点, 则该二叉搜索树的中序遍历序列单调非降  
3. 1. 二叉搜索树BST继承自前文介绍的二叉树类  
 [二叉树类](http://nbviewer.jupyter.org/github/lj72808up/datastruct/blob/master/bintree/1-BinaryTree.ipynb)

#### 二. BST关键码查找
1. BST查找  
  1. 从根节点开始, 按照关键码比较结果决定递归查找左子树或右子树  
  2. BST类的属性hot, 记录查找节点的父节点(该位置可作为后面插入节点到BST的接入位置)   
  3. BST查找, 一般将叶子节点的None孩子节点也看做"真实节点",便于递归  
2. 查找效率  
 BST的每一层, 算法至多访问1个节点, 只需常数时间. 然而不幸的是, 规模为n的BST, 其深度可能为n. 此时退化成一个有序列表, 此时的查找等效于顺序查找, 因此提高BST的搜索效率, 关键在于减小树的高度
 
#### 三. BST节点插入
1. 插入步骤  
  1. 首先, 利用BST查找算法, 找到插入位置及方向, 然后将新节点作为叶子节点插入  
  2. 从节点hot开始, 更新其父节点的高度  
  
#### 四. BST节点删除
节点删除的情况比较复杂, 需要分情况讨论待删除节点的状态, 找到能够替换掉该节点的节点  
1. 待删除节点只有单分支  
 当待删除节点只有单分支, 将该待删除节点替换为该节点的孩子节点即可  
2. 待删除节点有双分支  
 当待删除节点有双分支, 为保证BST的顺序定义(左孩子<节点<右孩子), 该节点的data属性应被替换为其右子树的最左节点的data, 因为右子树的最左节点必无左孩子, 所以删除该最左节点的情形同只有一个分支的情况

In [1]:
# -*- coding: UTF-8 -*-
from bintree import BinNode,BinTree,BinNodeUtil,BinTreeUtility

class BST(BinTree):
    def search(self,node,data,flag):
        '''从节点node开始查找关键码为data的节点'''
        if (node is None):  # 查找失败返回待查找节点出与左子树(-1)还是右子树(1)
            return (False,flag)
        if data==node.data: # 查找成功返回找到的节点
            return (True,node)
        # 记录下当前节点, 然后再递归查找
        self.hot = v
        if data<node.data:
            next_node = node.lc 
            flag = -1 # -1: 表示下次遍历在左子树
        else:
            next_node = node.rc
            flag = 1  # 1:表示下次遍历在右子树
        return self.search(next_node,data,flag)
    def search_from_bst(data):
        '''从BST中查找关键码为data的节点'''
        self.hot = None   # hot记录cha'zha
        return self.search_from_bst(self.root,data)
    
    def insert(self,data):
        flag,x = self.search_from_bst(data)
        if flag:
            return x
        else:
            new_node = BinNode(data,parent=self.hot)
            self.size = self.size + 1
            self.update_height_above(hot)
            if x == -1:
                self.hot.lc = new_node
            if x == 1:
                self.hot.rc = new_node
        return new_node
    
    def removeAt(self,node):
        '''删除节点:
           succ : 返回值,为实际被删除节点的接替者
           self.hot : 实际被删除节点的parent. 
                      单分支下hot为被删除节点的父节点
                      双分支下hot为右子树最左节点的parent 
        '''
        # 1. 判断分支个数
        if (node.lc is not None) and (node.rc is not None):
            branch_num = 2
        elif (node.lc is None) and (node.rc is None):
            branch_num = 0
        else:
            branch_num = 1
            
        # 2. 单分支的情况
        if branch_num == 1:
            succ = node.rc if not BinNodeUtil.has_lchild(node) else node.lc
            succ.parent = node.parent
            if (node.parent is not None) and (BinNodeUtil.is_lchild(node)):
                node.parent.lc = succ
            if (node.parent is not None) and (BinNodeUtil.is_rchild(node)):  
                node.parent.rc = succ
            if node.parent is None:
                self.root = succ

            self.hot = node.parent

        # 3. 双分支的情况
        if branch_num == 2: # 双分枝情况
            w = BinTreeUtility.succ(node) # 右子树最左节点
            # 交换节点data属性
            data = node.data
            w.data = node.data
            node.data = tmp
            # 接上右子树最左节点的parent和右子树最左节点的右子树
            u = w.parent
            if u is node:
                u.rc = w.rc
                succ = w.rc 
            else:
                u.lc = w.rc
                succ = w.rc
            self.hot = succ.parent
            
        # 4. 没有分支的情况